# Conversion to GeoDataFrame & Visualization with HoloViz

In [ ]:
import uxarray as ux
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import hvplot.pandas
import holoviews as hv

import warnings
warnings.filterwarnings("ignore")

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("Color_Theme", plt.cm.Blues(np.linspace(0.2, 1, 30)))

## Data

In this notebook, we will be using E3SM output, {elaborate more on the data}.

In [ ]:
base_path = "../../test/meshfiles/ugrid/outCSne30/"
grid_path = base_path + "outCSne30.ug"
data_path = base_path + "outCSne30_vortex.nc"

In [ ]:
uxds = ux.open_dataset(grid_path, data_path)

In [ ]:
uxds

In [ ]:
uxds.uxgrid

## Conversion to `spatialpandas.GeoDataFrame` for Visualization

In order to support visualization with the popular HoloViz stack of libraries (hvPlot, HoloViews, Datashader, etc.), UXarray provides methods for converting `Grid` and `UxDatArray` objects into a SpatialPandas `GeoDataFrame`, which can be used for visiualizaing the nodes, edges, and polygons the compose each grid, in addition to data variables.

TODO: Links to Ian's SIParCS work, brief overview of antimeridian handling

### `Grid` Conversion

In [ ]:
gdf_grid = uxds.uxgrid.to_geodataframe()
gdf_grid

### `UxDataArray` & `UxDataset` Conversion

In [ ]:
gdf_data = uxds['psi'].to_geodataframe()
gdf_data

### Challenges with Representing Geoscience Data as Geometries

When we convert to a `GeoDataFrame`, we internally represent the surface of a sphere as a collection of polygons over a 2D projection. However, this leads to issues around the Antimeridian (180 degrees east or west), which polygons are incorrectly constructed and have incorrect geometries. When constructing the `GeoDataFrame`, UXarray detects and corrects any polygon that touches or crosses the antimeridian. An array of indices of these faces can be accessed as part of the `Grid` object.
<br>

<figure>
<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Earth_map_with_180th_meridian.jpg/640px-Earth_map_with_180th_meridian.jpg" style="height: 250px; width:500px;"/></center>
<center><figcaption>Antimeridian Visual</figcaption></center>
</figure>


In [ ]:
uxds.uxgrid.antimeridian_face_indices

Taking a look at one of these faces that crosses or touches the antimeridian, we can see that it's split across the antimeridian and represented as a `MultiPolygon`, which allows us to properly render this two dimension grid.

In [ ]:
gdf_data.geometry[uxds.uxgrid.antimeridian_face_indices[0]]

In [ ]:
gdf_data.geometry[uxds.uxgrid.antimeridian_face_indices[0]].to_shapely()

For more details about the algorithm used for splitting these polygons, see the [Antimeridian Python Package](https://antimeridian.readthedocs.io/en/stable/).


## Visualizing Geometries

### Nodes

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"size": 6.0, "xlabel": "Longitude", "ylabel": "Latitude",
               "coastline": True, "width": 1600}

gdf_grid.hvplot.points(**plot_kwargs)

In [ ]:
hv.extension("bokeh")

plot_kwargs = {"s": 2.0, "xlabel": "Longitude", "ylabel": "Latitude", "coastline": True, "frame_width": 700}

gdf_grid.hvplot.points(**plot_kwargs)

### Edges

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"linewidth": 0.5, "xlabel":" Longitude", "ylabel": "Latitude", "coastline": True, "width": 1600}

gdf_grid.hvplot.paths(**plot_kwargs)

In [ ]:
hv.extension("bokeh")

plot_kwargs = {"line_width": 0.5, "xlabel": "Longitude", "ylabel": "Latitude", "coastline": True, "frame_width": 700}

gdf_grid.hvplot.paths(**plot_kwargs)

## Visualizing Data Variables

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"c": "psi", "cmap": cmap, "width": 400, "height": 200} #TODO

gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True)

add node about matplotlib vector not plotting properly, link to bokeh issue


In [ ]:
hv.extension("bokeh")

plot_kwargs = {"c": "psi",  "cmap": cmap, "line_width": 0.1,  "frame_width": 500, "frame_height": 250}

gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True)

hv.Layout(gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True) + gdf_data.hvplot.polygons(**plot_kwargs, rasterize=False)).cols(1)